In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [13]:
import os
os.chdir("/content/drive/MyDrive/CMPT733_Final_Project")
!ls

 covid_19_data.csv
 covid-19_global_daily.csv
 covid-19_global_daily.gsheet
 covid-19_us_daily.csv
 Covid-19_visualization.ipynb
 covid_tweet
 covid_tweet.csv
 data_analysis_visulization
'Final Presentation.gslides'
 Find_Other_Tags.ipynb
 graphs
 ieeeset_extraction.ipynb
 JKU_repo_ETL.ipynb
 Kevin_Milestone_Presentation.m4a
 media_accounts.txt
 Media_Analysis.ipynb
 milestone_presentation.docx
 Milestone_with_SFU_ppt_template.pptx
'New Recording 3.m4a'
 other_tags.csv
 preprocess_visualization_covid19_us_2.ipynb
 preprocess_visualization_covid19_us.ipynb
'Project Report.gdoc'
 Project_Timeline.xlsx
 project_twitter_gettingdata.ipynb
 Sentiment_Analysis_v1.ipynb
 Sentiment_Analysis_Visualization.ipynb
 SFU-Powerpoint-Template-Standard.pptx
 state-geocodes-v2016.xls
 Tag_Clustering.ipynb
 tag_score_prediction.ipynb
 tag_score_prediction_v2.ipynb
 tweet_sentiment_added
 tweet_sentiment_added_noise_removed
 us_counties_covid19_daily.csv
 us_covid19_daily.csv
 us_states_covid19_daily.csv


In [3]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import re
from datetime import datetime

In [4]:
# time series
confirmed_us = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
deaths_us = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')

confirmed_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_global = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [16]:
daily_us= pd.read_csv('covid-19_global_daily.csv')
daily_global = pd.read_csv('covid-19_global_daily.csv')

In [17]:
daily_global

,Unnamed: 0,Country/Region,Date,Comfirmed,Deaths,Recovered,Active,Daily_Increase
0,0,Afghanistan,1/22/20,0,0,0,0,0
1,1,Albania,1/22/20,0,0,0,0,0
2,2,Algeria,1/22/20,0,0,0,0,0
3,3,Andorra,1/22/20,0,0,0,0,0
4,4,Angola,1/22/20,0,0,0,0,0
...,...,...,...,...,...,...,...,...
85243,85243,Vietnam,4/9/21,2683,35,2429,219,15
85244,85244,West Bank and Gaza,4/9/21,264395,2812,229876,31707,2378
85245,85245,Yemen,4/9/21,5233,1022,1987,2224,100
85246,85246,Zambia,4/9/21,89783,1226,85559,2998,191


In [21]:
choro_map=px.choropleth(daily_global, 
                    locations="Country/Region", 
                    locationmode = "country names",
                    color="Comfirmed", 
                    hover_name="Country/Region", 
                    animation_frame="Date"
                   )

choro_map.update_layout(
    title_text = 'Global Spread of Coronavirus',
    title_x = 0.5,
    height = 800,
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
choro_map.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
daily_us

,Date,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,FIPS,Incident_Rate,Total_Test_Results,People_Hospitalized,Case_Fatality_Ratio,UID,ISO3,Testing_Rate,Hospitalization_Rate,Population,Daily_Increase,Daily_Tested,Daily_Pos_over_Tested,Confirmed_Percentage
0,2020-04-12,Alabama,US,2020-04-12 23:18:15,32.3182,-86.9023,3667,93,NaN,3470.0,1.0,75.988020,21583.0,437.0,2.610160,84000001.0,USA,460.300152,12.264945,4903185,3667,21583.0,16.990224,0.074788
1,2020-04-12,Alaska,US,2020-04-12 23:18:15,61.3707,-152.4044,272,8,66.0,264.0,2.0,45.504049,8038.0,31.0,2.941176,84000002.0,USA,1344.711576,11.397059,740995,272,8038.0,3.383926,0.036707
2,2020-04-12,American Samoa,US,NaN,-14.2710,-170.1322,0,0,NaN,0.0,60.0,0.000000,3.0,NaN,NaN,16.0,ASM,5.391708,NaN,55641,0,3.0,0.000000,0.000000
3,2020-04-12,Arizona,US,2020-04-12 23:18:15,33.7298,-111.4312,3542,115,NaN,3427.0,4.0,48.662422,52289.0,NaN,3.246753,84000004.0,USA,578.522286,NaN,7278717,3542,52289.0,6.773891,0.048662
4,2020-04-12,Arkansas,US,2020-04-12 23:18:15,34.9697,-92.3731,1280,27,367.0,1253.0,5.0,49.439423,19722.0,130.0,2.109375,84000005.0,USA,761.753354,10.156250,3017804,1280,19722.0,6.490214,0.042415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17801,2021-02-12,Virginia,US,2021-02-13 05:30:30,37.7693,-78.1700,544209,6966,42712.0,494531.0,51.0,6375.816163,5553955.0,NaN,1.280023,84000051.0,USA,65068.743916,NaN,8535519,3191,35220.0,9.060193,6.375816
17802,2021-02-12,Washington,US,2021-02-13 05:30:30,47.4009,-121.4905,327167,4675,NaN,322492.0,53.0,4296.409680,4840076.0,NaN,1.428934,84000053.0,USA,63560.656729,NaN,7614893,1008,25411.0,3.966786,4.296410
17803,2021-02-12,West Virginia,US,2021-02-13 05:30:30,38.4912,-80.9545,126887,2200,111584.0,13103.0,54.0,7080.166973,2068472.0,NaN,1.733826,84000054.0,USA,115418.657063,NaN,1792147,467,14322.0,3.260718,7.080167
17804,2021-02-12,Wisconsin,US,2021-02-13 05:30:30,44.2685,-89.6165,605785,6734,535627.0,63424.0,55.0,10404.325751,6454144.0,NaN,1.111616,84000055.0,USA,110849.586273,NaN,5822434,1102,29891.0,3.686728,10.404326


In [ ]:
def graph_covid_monthly_cases(df, month, colorFeature):
    fig = px.choropleth(df, locations='state', 
                        color=colorFeature,
                        color_continuous_scale="Oranges",
                        locationmode="USA-states",
                        scope="usa",
                        labels="state",
                        title= month + " 2020 COVID-19 " + colorFeature + " Cases")
    
    fig.update_layout(
        margin={"r":0,"t":0,"l":0,"b":0}, 
        title={'y':0.99,
            'x':0.48,
            'xanchor': 'center',
            'yanchor': 'top'
    })
    fig.show()